In [5]:
%reload_ext autoreload
%autoreload 2

from utils import load_dataset, SpecialTokens
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import numpy as np
import lightning.pytorch as pl

from models_2 import EncoderGRU, DecoderGRU, Seq2seq
from train import Trainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
train_size = 10000
val_size = 1000
train_data, human_vocab, machine_vocab = load_dataset(train_size)
val_data, _, _ = load_dataset(val_size)

100%|██████████| 1000/1000 [00:00<00:00, 42633.71it/s]


In [7]:
human_vocab

{';': 0,
 '?': 1,
 ' ': 2,
 '.': 3,
 '/': 4,
 '0': 5,
 '1': 6,
 '2': 7,
 '3': 8,
 '4': 9,
 '5': 10,
 '6': 11,
 '7': 12,
 '8': 13,
 '9': 14,
 'a': 15,
 'b': 16,
 'c': 17,
 'd': 18,
 'e': 19,
 'f': 20,
 'g': 21,
 'h': 22,
 'i': 23,
 'j': 24,
 'l': 25,
 'm': 26,
 'n': 27,
 'o': 28,
 'p': 29,
 'r': 30,
 's': 31,
 't': 32,
 'u': 33,
 'v': 34,
 'w': 35,
 'y': 36}

In [8]:
machine_vocab

{';': 0,
 '>': 1,
 '<': 2,
 '-': 3,
 '0': 4,
 '1': 5,
 '2': 6,
 '3': 7,
 '4': 8,
 '5': 9,
 '6': 10,
 '7': 11,
 '8': 12,
 '9': 13}

In [9]:
train_data[:20]

[('4/9/97', '>1997-04-09<'),
 ('9/8/83', '>1983-09-08<'),
 ('8/7/97', '>1997-08-07<'),
 ('4/2/02', '>2002-04-02<'),
 ('1/9/12', '>2012-01-09<'),
 ('7/3/09', '>2009-07-03<'),
 ('9/9/84', '>1984-09-09<'),
 ('2/2/00', '>2000-02-02<'),
 ('1/1/79', '>1979-01-01<'),
 ('4/7/92', '>1992-04-07<'),
 ('8/7/81', '>1981-08-07<'),
 ('8/3/80', '>1980-08-03<'),
 ('6/7/07', '>2007-06-07<'),
 ('7/3/09', '>2009-07-03<'),
 ('1/8/95', '>1995-01-08<'),
 ('1/2/71', '>1971-01-02<'),
 ('7/2/00', '>2000-07-02<'),
 ('8/4/98', '>1998-08-04<'),
 ('4/3/78', '>1978-04-03<'),
 ('7/3/95', '>1995-07-03<')]

In [10]:
val_data[:20]

[('4/7/24', '>2024-04-07<'),
 ('5/3/70', '>1970-05-03<'),
 ('8/2/77', '>1977-08-02<'),
 ('9/7/73', '>1973-09-07<'),
 ('2/9/97', '>1997-02-09<'),
 ('7/3/96', '>1996-07-03<'),
 ('1/1/73', '>1973-01-01<'),
 ('6/30/82', '>1982-06-30<'),
 ('8 06 22', '>2022-06-08<'),
 ('8/28/98', '>1998-08-28<'),
 ('8/11/96', '>1996-08-11<'),
 ('4/10/89', '>1989-04-10<'),
 ('9 06 94', '>1994-06-09<'),
 ('4/28/93', '>1993-04-28<'),
 ('2/17/02', '>2002-02-17<'),
 ('6 04 98', '>1998-04-06<'),
 ('7/22/10', '>2010-07-22<'),
 ('9/30/72', '>1972-09-30<'),
 ('1/18/84', '>1984-01-18<'),
 ('2/13/92', '>1992-02-13<')]

In [11]:
class Lang:
    def _get_char(self, ind):
        if isinstance(ind, torch.Tensor):
            return self.inv_vocab[ind.item()]
        else:
            return self.inv_vocab[ind]

    def __init__(self, vocab: dict):
        self.vocab = vocab
        self.inv_vocab = {v:k for k,v in vocab.items()}
        self.vocab_size = len(vocab)

    def str_to_ind(self, str):
        return [self.vocab[c] for c in str]
    
    def ind_to_str(self, ind):
        return ''.join([self._get_char(i) for i in ind])

In [12]:
test = Lang(human_vocab)
date = train_data[0][0]
print(date)
translated_date = test.str_to_ind(date)
print(translated_date)
reversed_translation = test.ind_to_str(translated_date)
print(reversed_translation)

4/9/97
[9, 4, 14, 4, 14, 12]
4/9/97


In [13]:
class TranslationTrainingDataset(Dataset):
    def __init__(self, data, input_vocab, output_vocab):
        self.input_lang = Lang(input_vocab)
        self.target_lang = Lang(output_vocab)

        self.data = data

        self.encoder_inputs = [self.input_lang.str_to_ind(input_sent) for input_sent, _ in self.data]

        targets = [self.target_lang.str_to_ind(target_sent) for _, target_sent in self.data]
        self.decoder_inputs = [target[:-1] for target in targets]
        self.decoder_targets = [target[1:] for target in targets]

    def __getitem__(self, index):
        return self.encoder_inputs[index], self.decoder_inputs[index], self.decoder_targets[index]
    
    def __len__(self):
        return len(self.encoder_inputs)


In [14]:
train_dataset = TranslationTrainingDataset(train_data, human_vocab, machine_vocab)
val_dataset = TranslationTrainingDataset(val_data, human_vocab, machine_vocab)

In [15]:
x,y,z = train_dataset[0]
print(x, train_dataset.input_lang.ind_to_str(x))
print(y, train_dataset.target_lang.ind_to_str(y))
print(z, train_dataset.target_lang.ind_to_str(z))

[9, 4, 14, 4, 14, 12] 4/9/97
[1, 5, 13, 13, 11, 3, 4, 8, 3, 4, 13] >1997-04-09
[5, 13, 13, 11, 3, 4, 8, 3, 4, 13, 2] 1997-04-09<


In [16]:
def collate_batch(data):
    batch = []
    for i in range(len(data[0])):
        batch_data = [torch.tensor(item[i], dtype=torch.int64) for item in data]
        batch_data = nn.utils.rnn.pad_sequence(batch_data, batch_first=True)
        batch.append(batch_data)


    return tuple(batch)

In [17]:
train_loader = DataLoader(dataset=train_dataset, collate_fn=collate_batch, batch_size = 64, num_workers = 8)
val_loader = DataLoader(dataset=val_dataset, collate_fn=collate_batch, batch_size = 64, num_workers = 8)

In [20]:
x_enc_batch,  x_dec_batch, y_batch = next(iter(train_loader))
# print(x_enc_batch)
# print(x_dec_batch)
# print(y_batch)
encoder = EncoderGRU(len(human_vocab), hidden_size=64, embedding_dim=32, num_layers=1, bidirectional=False)
print(encoder)
enc_out, enc_hidden = encoder(x_enc_batch)
print(enc_out.shape, enc_hidden.shape)

EncoderGRU(
  (embedding): Embedding(37, 32)
  (gru): GRU(32, 64, batch_first=True)
)
torch.Size([64, 6, 64]) torch.Size([1, 64, 64])


In [21]:
lang = train_dataset.input_lang
for row in x_enc_batch:
    print(lang.ind_to_str(row))

4/9/97
9/8/83
8/7/97
4/2/02
1/9/12
7/3/09
9/9/84
2/2/00
1/1/79
4/7/92
8/7/81
8/3/80
6/7/07
7/3/09
1/8/95
1/2/71
7/2/00
8/4/98
4/3/78
7/3/95
8/5/91
8/9/83
8/5/04
8/5/18
9/4/14
1/3/21
7/9/97
6/3/88
2/2/84
4/2/82
6/2/97
9/2/18
2/1/07
3/5/70
3/2/07
4/7/21
7/7/99
6/9/87
5/7/14
2/5/71
9/1/92
5/2/00
8/1/74
2/9/05
6/6/07
5/6/93
1/5/11
3/7/90
6/4/94
1/5/10
5/8/21
7/2/11
6/3/91
3/4/76
8/4/07
2/8/23
7/6/73
2/5/19
9/5/04
4/5/71
8/1/01
3/7/80
2/2/13
6/1/02


In [22]:
len(machine_vocab)

14

In [24]:
encoder = EncoderGRU(len(human_vocab), hidden_size=64, embedding_dim=32, num_layers=1, bidirectional=False)
decoder = DecoderGRU(len(machine_vocab), hidden_size=64, embedding_dim=32, num_layers=1)
print(decoder)
print(encoder)

DecoderGRU(
  (embedding): Embedding(14, 32)
  (gru): GRU(32, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=14, bias=True)
)
EncoderGRU(
  (embedding): Embedding(37, 32)
  (gru): GRU(32, 64, batch_first=True)
)


In [25]:
print("Decoder forward pass\n")

# Teacher forcing
print("Training with teacher forcing")
print(f"Input batch shape: {x_dec_batch.shape}")
dec_out, dec_hid = decoder(x_dec_batch, enc_hidden)
print(f"decoder output shape: {dec_out.shape}\ndecoder hn shape: {dec_hid.shape}")
# loss(dec_out, target)
print()
# Without teacher forcing
print("Training without teacher forcing")
dec_input = x_dec_batch[:,0:1]
print(f"Decoder 1st input shape: {dec_input.shape}")
dec_out, dec_hid = decoder(dec_input, enc_hidden)
print(f"decoder output shape: {dec_out.shape}\ndecoder hn shape: {dec_hid.shape}")
next_input = torch.argmax(dec_out, dim=-1)
print(f"decoder 2nd input shape: {next_input.shape}")


Decoder forward pass

Training with teacher forcing
Input batch shape: torch.Size([64, 11])
decoder output shape: torch.Size([64, 11, 14])
decoder hn shape: torch.Size([1, 64, 64])

Training without teacher forcing
Decoder 1st input shape: torch.Size([64, 1])
decoder output shape: torch.Size([64, 1, 14])
decoder hn shape: torch.Size([1, 64, 64])
decoder 2nd input shape: torch.Size([64, 1])


In [26]:
model = Seq2seq(encoder, decoder)
model

Seq2seq(
  (encoder): EncoderGRU(
    (embedding): Embedding(37, 32)
    (gru): GRU(32, 64, batch_first=True)
  )
  (decoder): DecoderGRU(
    (embedding): Embedding(14, 32)
    (gru): GRU(32, 64, batch_first=True)
    (fc): Linear(in_features=64, out_features=14, bias=True)
  )
)

In [27]:
print("Model forward pass (input -> encoder -> decoder -> output)\n")

# Teacher forcing
print("Training without teacher forcing (out_length = 20)")
print(f"Input batch shape: {x_enc_batch.shape}")
output =  model(x_enc_batch, out_length = 20)
print(f"Output shape: {output.shape}")

print()
# Without teacher forcing
print("Training with teacher forcing")
print(f"Encoder input batch shape: {x_enc_batch.shape}")
print(f"Decoder input batch shape: {x_dec_batch.shape}")
output = model(x_enc_batch, dec_input_batch = x_dec_batch, teacher_forcing = True)
print(f"Output shape: {output.shape}")

Model forward pass (input -> encoder -> decoder -> output)

Training without teacher forcing (out_length = 20)
Input batch shape: torch.Size([64, 6])
Output shape: torch.Size([64, 20, 14])

Training with teacher forcing
Encoder input batch shape: torch.Size([64, 6])
Decoder input batch shape: torch.Size([64, 11])
Output shape: torch.Size([64, 11, 14])


In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
cross_entropy_loss = nn.CrossEntropyLoss(ignore_index=0)

trainer = Trainer(model, train_dataLoader=train_loader, val_dataLoader=val_loader, loss_fn= cross_entropy_loss, optimizer=optimizer)

In [29]:
trainer.train(50)

Epoch  1    training loss: 1.764870 | validation loss: 1.473958
Epoch  2    training loss: 1.107737 | validation loss: 0.959201
Epoch  3    training loss: 0.822331 | validation loss: 0.851810
Epoch  4    training loss: 0.719037 | validation loss: 0.766839
Epoch  5    training loss: 0.659734 | validation loss: 0.711376
Epoch  6    training loss: 0.632986 | validation loss: 0.691685
Epoch  7    training loss: 0.616020 | validation loss: 0.684957
Epoch  8    training loss: 0.606244 | validation loss: 0.689338
Epoch  9    training loss: 0.600491 | validation loss: 0.679058
Epoch  10   training loss: 0.598975 | validation loss: 0.673430
Epoch  11   training loss: 0.590169 | validation loss: 0.657714
Epoch  12   training loss: 0.576176 | validation loss: 0.629070
Epoch  13   training loss: 0.567702 | validation loss: 0.618632
Epoch  14   training loss: 0.559543 | validation loss: 0.605172
Epoch  15   training loss: 0.543017 | validation loss: 0.572176
Epoch  16   training loss: 0.519836 | va

In [30]:
def print_translations(model, n = 10):
    test_loader = DataLoader(dataset=val_dataset, collate_fn=collate_batch, batch_size = 64, num_workers = 8, shuffle=True)

    x, z, y = next(iter(test_loader))
    x = x.to(device)
    y = y.to(device)
    z = z.to(device)

    y_hat = model(
            x,
            teacher_forcing=False,
            sos_index=1,
            out_length=11,
        )

    input_lang = train_dataset.input_lang
    output_lang = train_dataset.target_lang

    y_hat = y_hat.argmax(axis=-1)

    print("Translation test\n")
    print("Input                 |   Machine translation | Correct translation")
    for i in range(10):
        print(input_lang.ind_to_str(x[i]).replace(';', ' '), output_lang.ind_to_str(y_hat[i]).strip('<>;'), " "*10, output_lang.ind_to_str(y[i]).strip(';<>'))

In [31]:
print_translations(model)

Translation test

Input                 |   Machine translation | Correct translation
4 nov 1978                  1978-11-24            1978-11-04
26 feb 1978                 1978-03-22            1978-02-26
tuesday november 11 2003    2003-11-11            2003-11-11
15 nov 1994                 1994-11-10            1994-11-15
18 may 2011                 2011-01-28            2011-05-18
06 feb 1981                 1982-12-06            1981-02-06
26 apr 1985                 1985-04-26            1985-04-26
friday april 7 2017         2017-04-07            2017-04-07
3 august 1992               1992-08-03            1992-08-03
thursday june 29 2023       2023-06-29            2023-06-29


In [52]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)
        
        print(scores.shape)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

In [74]:
hidden_size = 32
output_size = len(machine_vocab)
batch_size = 5


att = BahdanauAttention(hidden_size)

hidden = torch.zeros(1, batch_size, hidden_size)
keys = torch.zeros(batch_size, output_size, hidden_size) # encoder outputs

query = hidden.permute(1, 0, 2)

c, w = att(query, keys)
print(c.shape, w.shape)

torch.Size([5, 1, 14])
torch.Size([5, 1, 32]) torch.Size([5, 1, 14])


In [ ]:
class AttentionDecoderGRU(nn.Module):
    def __init__(self, hidden_size, vocab_size):
        super(AttentionDecoderGRU, self).__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, vocab_size)

    def forward(self, encoder_outputs, encoder_hidden, sos_index = 1, decoder_input = None, teacher_forcing = False, out_length = 1):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(sos_index)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(out_length):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if teacher_forcing:
                # Teacher forcing: Feed the target as the next input
                decoder_input = decoder_input[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        # input_gru = torch.cat((embedded, context), dim=2)
        input_gru = context

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [ ]:
class AttentionSeq2seq(nn.Module):
    def __init__(self, encoder: EncoderGRU, decoder: AttentionDecoderGRU) -> None:
        super(Seq2seq, self).__init__()

        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_input_batch, sos_index = 1, decoder_input = None, teacher_forcing = False, out_length = 1):
        encoder_outputs, encoder_hidden = self.encoder(enc_input_batch)
        batch_size = len(enc_input_batch)
        
        decoder_output, _, _ = decoder(encoder_outputs, encoder_hidden, sos_index, decoder_input, teacher_forcing, out_length)
        
        return decoder_output